In [4]:
import os
import requests
from lxml import etree
import io

def fetch_xsd_files_with_content(project_path):
    xsd_files_with_content = []

    for root, dirs, files in os.walk(project_path):
        for file in files:
            if file.endswith('.xsd'):
                xsd_path = os.path.join(root, file)
                with open(xsd_path, 'r') as xsd_file:
                    xsd_content = xsd_file.read()
                    xsd_files_with_content.append((file, xsd_content))

    return xsd_files_with_content

def fetch_xml_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching XML file from {url}: {e}")
        return None

def is_valid_unit_operation(xsd_content, unit_operation_xml):
    try:
        # Parse the .xsd content
        xsd_schema = etree.XMLSchema(etree.fromstring(xsd_content.encode()))  # Convert to bytes and parse
        # Parse the UnitOperation XML
        unit_operation_root = etree.XML(unit_operation_xml.encode())

        # Validate the UnitOperation XML against the .xsd schema
        return xsd_schema.validate(unit_operation_root)
    except etree.XMLSyntaxError:
        return False

def fetch_xsd_content(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Check for any HTTP errors
        return response.text
    except requests.exceptions.RequestException as e:
        print(f"Error fetching .xsd file from {url}: {e}")
        return None

# Local path to the cloned project repla
project_path = '/path/to/your/cloned/project'

# Fetch all .xsd files and XML content locally
all_xsd_files_with_content = fetch_xsd_files_with_content(project_path)

url_to_xml = "https://raw.githubusercontent.com/Gressling/S88/main/Sequences/Example/3UnitOperations.xml"
xml_content = fetch_xml_from_url(url_to_xml)

xsd_url = "https://raw.githubusercontent.com/Gressling/S88/main/Sequences/Sequences.xsd"
xsd_content = fetch_xsd_content(xsd_url)

if xml_content is not None:
    all_unit_operations_valid = True
    for event, element in etree.iterparse(io.BytesIO(xml_content.encode()), events=('end',), tag='UnitOperation'):
        tag_after_unit_operation = element.getchildren()[0].tag
        unit_operation_valid = False
        for xsd_filename, xsd_content in all_xsd_files_with_content:
            if is_valid_unit_operation(xsd_content, etree.tostring(element).decode()):
                unit_operation_valid = True
                break
        if not unit_operation_valid:
            all_unit_operations_valid = False
            print(f"Unit Operation <{tag_after_unit_operation}> is not valid.")
        element.clear()  # Clear the element from memory after processing

    if xml_content and xsd_content:
        if all_unit_operations_valid:
            print("The Sequence is valid with its Unit Operations.")
        else:
            print("The Sequence is false but the UnitOperations are true.")
    else:
        if xml_content is not None:
            print("The Sequence is true but the UnitOperations are not valid.")
        else:
            print("The Sequence and the UnitOperations are false.")


The Sequence is valid with its Unit Operations.
